<a href="https://colab.research.google.com/github/amberaeschbach/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,244 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [983 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:13 https://ppa.launchpadcont

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# Add the file to Spark using SparkFiles
spark.sparkContext.addFile(url)

In [6]:
# Read the CSV file into a DataFrame
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [8]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("my_table")
print(home_sales_df)

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]


In [9]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 4 GROUP BY date ORDER BY date")
result.show()


+----------+-------------+
|      date|average_price|
+----------+-------------+
|2019-01-01|     365088.0|
|2019-01-02|    271644.29|
|2019-01-03|    381218.86|
|2019-01-04|    359586.14|
|2019-01-05|    323683.57|
|2019-01-06|    301091.13|
|2019-01-07|     327474.5|
|2019-01-08|    304567.83|
|2019-01-09|     243619.0|
|2019-01-10|     287844.4|
|2019-01-11|    277574.29|
|2019-01-12|     218507.0|
|2019-01-13|    294603.86|
|2019-01-14|    284434.57|
|2019-01-15|     254288.5|
|2019-01-16|     335106.5|
|2019-01-17|    333439.09|
|2019-01-18|    297542.56|
|2019-01-19|    294484.22|
|2019-01-20|    312395.33|
+----------+-------------+
only showing top 20 rows



In [10]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
result.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [11]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM my_table WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
result.show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM my_table WHERE price >= 350000")
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.47393798828125 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('my_table')

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('my_table')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.507469177246094e-05 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output = "path/to/partitioned_data.parquet"

# Write the DataFrame to Parquet format with partitioning
home_sales_df.write.partitionBy("date_built").parquet(output)

In [21]:
# 11. Read the parquet formatted data.
parquet = "path/to/partitioned_data.parquet"

parquet_data = spark.read.parquet(parquet)
parquet_data

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [22]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
    SELECT ROUND(AVG(view), 2) AS average_view
    FROM my_table
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.34171080589294434 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("my_table")

In [29]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("my_table")

if is_cached:
    print("The 'my_table'  is still cached.")
else:
    print("The 'my_table'  is no longer cached.")



The 'my_table'  is no longer cached.
